## Parsing historical and current legislator data for FEC ID: out to csv (histcurrent.merged.csv)

In [1]:
import yaml
import pandas as pd
from pandas.io.json import json_normalize

In [2]:
def read_yaml(fn):
    with open(fn, 'r') as fi:
        return yaml.load(fi)

def filter_data(data):
    result_data = []
    for x in data:
        if 'id' not in x:   continue
        if 'fec' not in x['id']:    continue
        if 'thomas' not in x['id']: continue
        result_data.append(x)
    return result_data


fn = 'legislators-historical.yaml'


df = json_normalize(filter_data(read_yaml(fn)), 'terms', [['id', 'fec',], ['id', 'thomas']])

df['id.fec'] = df['id.fec'].map(lambda x: str(x)[:-2])


df['id.fec'] = df['id.fec'].map(lambda x: str(x)[2:])



In [3]:
def read_yaml(fm):
    with open(fm, 'r') as fi:
        return yaml.load(fi)
def filter_data(data):
    result_data = []
    for x in data:
        if 'id' not in x:   continue
        if 'fec' not in x['id']:    continue
        if 'thomas' not in x['id']: continue
        result_data.append(x)
    return result_data
fm = 'legislators-current.yaml'
dy = json_normalize(filter_data(read_yaml(fm)), 'terms', [['id', 'fec',], ['id', 'thomas']])
dy['id.fec'] = dy['id.fec'].map(lambda x: str(x)[:-2])

dy['id.fec'] = dy['id.fec'].map(lambda x: str(x)[2:])

In [4]:
merged = [df,dy]
tocsv = pd.concat(merged)

tocsv.to_csv('histcurrent.merged.csv', index=False)


## Creating FEC and Thomas ID index: send to csv (fecthomas.mrg)

In [5]:
fecthomasmerge=pd.read_csv('histcurrent.merged.csv', \
index_col=0, usecols=['id.thomas', 'id.fec'])

In [6]:
fecthomasmerge = fecthomasmerge.drop_duplicates('id.thomas', keep='first')

In [7]:
fecthomasmerge=fecthomasmerge.reset_index()

In [8]:
fecthomasmerge.to_csv('fecthomas.mrg', index=False)

In [9]:
fecthomasmerge.head(1)

,id.fec,id.thomas
0,S4TN00153,1336


In [10]:
import pandas as pd
import glob

In [11]:
df = pd.concat((pd.read_csv(f, names=['date','bills','id.thomas']) for f in glob.glob\
                ('/home/jayaramdas/anaconda3/dfs/*1**_s_ba')))

df.count()

date         51604
bills        51604
id.thomas    51604
dtype: int64

In [12]:
df['id.thomas'].nunique()

902

In [13]:
l = pd.read_csv(('fecthomas.csv'), usecols=['id.thomas', 'id.fec'])
dl = l.drop_duplicates('id.thomas', keep='first')

dl.count()

id.fec       915
id.thomas    915
dtype: int64

In [14]:
d = pd.merge(df, dl, on = 'id.thomas', how='inner')
df = d.drop_duplicates('bills', keep='first')
df.count()

date         50456
bills        50456
id.thomas    50456
id.fec       50456
dtype: int64

In [15]:
d['bills'].nunique()

50456

In [16]:
d['date'] = \
d['date'].map(lambda x: str(x)[:-6]) 

In [17]:
d.set_index((d['date']), inplace=True)

## Deriving the total number of bills sponsored per legislator: send to csv (billspon.legisl.merg)

In [18]:
d['leg_totalbills']=d.groupby(['date', 'id.thomas']).bills.transform('nunique')

In [19]:
del d['date']
d=d.reset_index()

In [20]:
d.head()

,date,bills,id.thomas,id.fec,leg_totalbills
0,2008,s2952-110,791,H2NJ13075,29
1,2007,s328-110,791,H2NJ13075,34
2,2007,s2528-110,791,H2NJ13075,34
3,2007,s2397-110,791,H2NJ13075,34
4,2007,s1603-110,791,H2NJ13075,34


In [21]:
d.to_csv('billspon.legisl.merg', mode = 'w', index=False)